In [159]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
response ="COVID-19 - PESQUISA DE ANTICORPOS IgG COVID-19 IgG"

In [160]:
pacientes = pd.read_csv(r'../Data/HC_PACIENTES_1.csv', sep='|')

df = pd.read_csv(r'../Data/HC_EXAMES_1.csv', sep='|')
df = pd.merge(df, pacientes)

df.insert(1, "EXAME", df['DE_EXAME']+" "+df['DE_ANALITO'])

out = df.pivot_table(index=['ID_aTENDIMENTO', 'IC_SEXO', 'AA_NASCIMENTO'], columns='EXAME', values='DE_RESULTADO', aggfunc='first')
out = out.reset_index(level=['IC_SEXO', 'AA_NASCIMENTO'])

In [161]:
convertedResult = pd.get_dummies(out, dummy_na=True)

In [162]:

def try_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return np.nan
le = LabelEncoder()

# Fit and transform the column
out['IC_SEXO'] = le.fit_transform(out['IC_SEXO'])
out.dropna(subset=[response], inplace=True)
#0 negativo,1 positivo
out = out[out[response].isin(["Não reagente", "Reagente"])]
out[response] = le.fit_transform(out[response])

# Apply to each value in the DataFrame
converted_df = out.applymap(try_float)
converted_df.dropna(axis=1, how='all')
for col in converted_df.columns:
    n_unique = converted_df[col].nunique(dropna=True)
    fill_value = 0.5 if n_unique == 2 else 0
    converted_df[col] = converted_df[col].fillna(fill_value)

/var/folders/bx/s_m61_b15zg60nmz229ccs0c0000gn/T/ipykernel_44120/2616678637.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  converted_df = out.applymap(try_float)


In [163]:
converted_df

EXAME,IC_SEXO,AA_NASCIMENTO,25-HIDROXIVITAMINA D 25-HIDROXIVITAMINA D,ADENOSINA DEAMINASE ADENOSINA DEAMINASE,ADENOVIRUS ADENOVIRUS,AGNOR AGNOR,AGREGAÇÃO PLAQUETÁRIA ADRENALINA % aos 10 minutos:,AGREGAÇÃO PLAQUETÁRIA ADRENALINA 1ª onda:,AGREGAÇÃO PLAQUETÁRIA ADRENALINA 2ª onda:,AGREGAÇÃO PLAQUETÁRIA ADRENALINA Resposta inicial:,...,VDRL VDRL,VELOCIDADE DE HEMOSSEDIMENTAÇÃO VELOCIDADE DE HEMOSSEDIMENTAÇÃO,VITAMINA B 12 VITAMINA B 12,VLDL - COLESTEROL VLDL - COLESTEROL,VORICONAZOL VORICONAZOL,VÍRUS SINCICIAL RESPIRATÓRIO VÍRUS SINCICIAL RESPIRATÓRIO,Vírus sincicial respiratório A (HRSVA) e B (HRSVB) Vírus sincicial respiratório A (HRSVA) e B (HRSV,ÁCIDO FÓLICO ÁCIDO FÓLICO,ÁCIDO VALPRÓICO ÁCIDO VALPRÓICO,ÁCIDO ÚRICO ÁCIDO ÚRICO
ID_aTENDIMENTO,,,,,,,,,,,,,,,,,,,,,
016586ae084a86326b913fe74146b102,1.0,1979.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
018c7e14ec0445c170461b744bd7e026,1.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0
0200a7fee74490fcac31fba3c1c3b282,0.0,1985.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02a99cab514cf42129c3bbc01a53b3a3,0.0,1962.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0
033a501926526b396c8672c83a8834f2,1.0,1958.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe946def4ae62a5f5bfacecb21925f04,1.0,1958.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fec10adce472163aba4474ca9b21be33,0.0,1997.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ff97502613f7d78596bb11f03a5d804d,0.0,1977.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
y = converted_df[response]
X = converted_df.drop(columns=[response])
# 
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
foldsScore = []
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    foldsScore.append(score)

score = np.mean(foldsScore)
print(f"average score: {score:.4f}")



average score: 0.6422
